In [1]:
load("WeightedAutomaton.py")
# Load database of witnesses for PFA complexity, so we have some more examples to play with later
SW = WeightedAutomaton._loadwits("witnesses")

In [2]:
# Create a WeightedAutomaton over the alphabet ['0','1','2'].
# First, the transition matrices. 
mytransitions = {'0': matrix([[1/2,0,1], [0,1/2,0], [0,0,1]]),
                 '1': matrix([[0,1,0], [1/8,0,1/2], [0,0,1]]),
                 '2': matrix([[1,1,0], [0,0,0], [0,0,1]])}
# Next, the initial and accepting state vectors
initialstates = [1/2,1/2,0]
acceptingstates = [1,0,0]
# Put these together into a WeightedAutomaton and pretty-print it
myWA = WeightedAutomaton(mytransitions,initialstates,acceptingstates)
myWA.show()

0


\(\left(\begin{array}{rrr} \frac{1}{2} & 0 & 1 \\ 0 & \frac{1}{2} & 0 \\ 0 & 0 & 1 \end{array}\right)\)


1


\(\left(\begin{array}{rrr} 0 & 1 & 0 \\ \frac{1}{8} & 0 & \frac{1}{2} \\ 0 & 0 & 1 \end{array}\right)\)


2


\(\left(\begin{array}{rrr} 1 & 1 & 0 \\ 0 & 0 & 0 \\ 0 & 0 & 1 \end{array}\right)\)


Initial state distribution: [1/2, 1/2, 0]
Final state distribution: [1, 0, 0]


In [3]:
# Or, we can manually retrieve some basic properties of the automaton:
print(myWA.size) # number of states
print(myWA.alphabet) # underlying alphabet the WA reads from. These must always be strings.
print(myWA.ring) # ring in which the entries of the transition matrices live
print(myWA.states,'\n') # list of names for the states. So far they can only be numbers (allowing arbitrary labels is planned).
print(myWA.transitions,'\n') # dictionary of transition matrices, exactly as given to the constructor
print(myWA.initial_states,'\n') # initial state distribution (row vector)
print(myWA.accepting_states) # final state distribution (column vector)

3
['0', '1', '2']
Rational Field
[0, 1, 2] 

{'0': [1/2   0   1]
[  0 1/2   0]
[  0   0   1], '1': [  0   1   0]
[1/8   0 1/2]
[  0   0   1], '2': [1 1 0]
[0 0 0]
[0 0 1]} 

[1/2 1/2   0] 

[1]
[0]
[0]


In [4]:
# This particular automaton is not stochastic, i.e., a PFA as defined by Rabin (or really the slight generalization
# which allows any stochastic vector as the initial state distribution):
myWA.is_pfa()

False

In [5]:
# This WA happens to have a "dead state", i.e., a nonaccepting state with no out-transitions
myWA.has_dead_state()

True

In [6]:
# We can check each state individually to see if it's dead. In this case only state 2 is
myWA.is_dead_state(2)

True

In [7]:
# Compute acceptance probability of the string 0200121
myWA.prob('0200121')

1/1024

In [8]:
# You can call myWA to get the same thing
myWA('0200121')

1/1024

In [9]:
# Display the 'probability' of going from state 0 to state 2 reading 0200121
myWA.trans_prob(0,2,'0200121')

233/128

In [10]:
# Show how the states of the automaton are weighted after the string is read, given as a row vector
myWA.read('0200121')

[ 1/1024   1/128 233/256]

In [11]:
# Compute its gap wrt myWA, that is, the minimum difference between the probs of 0200101 and of all other strings of the same length
myWA.gap('0200121')
# the value being negative means there are higher-probability strings of length 7.

-511/1024

In [12]:
# Another way to see that this string isn't the most likely of length 7 is the following function, which is faster
# since it immediately returns False when it sees a string with at least the same probability:
myWA.is_highest('0200121')

False

In [13]:
# If you're computing a lot of gaps over the same alphabet, you can pass in the precomputed set of all strings of a particular 
# length in order to save some runtime:
strings7 = myWA.strings([7])
myWA.gap('0200121',strings7)
# This is mainly useful when doing brute-force calculations over large numbers of WAs.

-511/1024

In [14]:
# You can do the same with is_highest() to save time:
myWA.is_highest('0200121',strings7)

False

In [15]:
# If you don't care about non-positive gaps, you can set the optional parameter cutoff=True in gap() to immediately
# return 0 if there is another string having at least the same probability. This saves time when testing lots of strings.
myWA.gap('0200121',cutoff=True)

0

In [16]:
# Find acceptance probabilities of all strings of length 5
# The output is a ProbabilityList, a dict in the format 'string':probability.
probs5 = myWA.probs_of_length(5)
# Find the highest-probability string(s) of that length
probs5.highest_prob()

[1/2, '22222']

In [17]:
# Find the gap between the two highest-prob strings listed in probs5. Output is a list of the form [gap, highest, second-highest].
probs5.highest_gap()

[1/4, '22222', '02222']

In [18]:
# Now do the same for all strings of lengths 4 through 7. Here we first generate a list of all such strings
# to pass to myWA.probs():
strings47 = myWA.strings(range(4,8))
probs47 = myWA.probs(strings47)
# Find the highest-probability string(s) among all those lengths. This time there are several sharing the same probability:
probs47.highest_prob()

[1/2, '2222', '22222', '222222', '2222222']

In [19]:
# Find the gap between the top two strings of length 7:
probs47.highest_gap(7)
# You can also pass a length argument to highest_prob() to find the highest-prob string(s) of just that length.

[1/4, '2222222', '0222222']

In [20]:
# The gap between the top two strings of /all/ lengths is 0 because 2^n has probability 1/2 for each n:
probs47.highest_gap()

[0, '2222', '22222']

In [21]:
# Let's see how many strings in the witness database are given a gap of more than 1/10 by a witness in the database.
# First, make the set of binary strings of each length in advance to save runtime:
S2 = {}
maxlength = 7 # let's keep this to a manageable length
for i in range(maxlength+1):
    # any WA reading from a binary alphabet will do for this. We do want to stratify by length however
    S2[i] = SW['0101'][0].strings([i])

desiredgap = 1/10
gapwitnesses = [] # this will contain the results of our search
for s in SW.keys(): # keys are binary strings; values are lists of WAs giving s a positive gap
    if len(s) > maxlength: continue
    for A in SW[s]: # for each PFA listed, see how big its gap is
        g = A.gap(s,S2[len(s)])
        if g >= desiredgap:
            gapwitnesses.append([s,g,A]) # add to our list
print(len(gapwitnesses))
# this is specifically the set of distinct strings represented
print(set([gw[0] for gw in gapwitnesses]))

13
{'0101', '1010', '10000'}


In [22]:
# Examine a witness to A_P(0110001) = 3 and see what its highest-prob strings are of a bunch of lengths:
W = SW['0110001'][0]
for l in range(1,13):
    print(W.probs_of_length(l).highest_gap(numerical=True))

[0.000000000000000, '0', '1']
[0.500000000000000, '01', '11']
[0.250000000000000, '101', '001']
[0.125000000000000, '0001', '1001']
[0.0625000000000000, '01101', '10001']
[0.0625000000000000, '010101', '000001']
[0.0156250000000000, '0110001', '1000001']
[0.0156250000000000, '01010001', '00000001']
[0.00390625000000000, '011000001', '100000001']
[0.00390625000000000, '0101000001', '0000000001']
[0.000976562500000000, '01100000001', '10000000001']
[0.000976562500000000, '010100000001', '000000000001']


In [23]:
# Let's play around with this example. First, create a copy since we don't want to accidentally mess up witnesses.pickle
W1 = deepcopy(W)
# switch the first and third states (has no effect on acceptance probabilities)
W1.swap_states(0,2)
# The read() function returns a row vector describing the state distribution after reading the specified word.
# Set W1's initial state vector to that obtained after reading 011000:
W1.initial_states = W1.read('011000')
# Demonstrate that the gap of an extension of 011000 wrt W1 is at least what it would have been for W,
# and that the inequality can be strict (see Proposition 3.3 in Gill 2024)
W.gap('011000001') < W1.gap('001')

True

In [24]:
# Change the probability of going from state 2 to state 0 reading '1' to 1/3
W1.set_transition(2,0,'1',1/3)
# Actually, let's change it to 2/3, and this time let's rescale the other out-transitions from state 1 reading '1'
# so that altogether they sum to 1:
W1.set_transition(2,0,'1',2/3,reweight=True)
# Do the same for the initial state distribution, setting the weight of state 0 to -1/2
W1.set_initial(0,-1/2,reweight=True)
# See how all this has changed the highest-probability strings
for l in range(1,13):
    print(W1.probs_of_length(l).highest_gap(numerical=True))

[1.59375000000000, '1', '0']
[0.489583333333333, '01', '10']
[0.0694444444444444, '111', '001']
[0.126736111111111, '1101', '0001']
[0.0604745370370370, '01101', '11001']
[0.0140817901234568, '111101', '110001']
[0.0419319058641975, '1101101', '0110001']
[0.00972141846707819, '01101101', '11110001']
[0.0104829764660494, '110110001', '011000001']
[0.000707639424725652, '1101101101', '0110110001']
[0.00262074411651235, '11011000001', '01100000001']
[0.000176909856181413, '110110110001', '011011000001']


In [25]:
# Create 3-state automaton over SR using the generators of a polynomial ring, aside from the final state vector
# which we'll fix at [1,0,0].
R = PolynomialRing(QQ,12,'q')
q = R.gens()
P0 = matrix(SR,2,2,q[0:4])
P1 = matrix(SR,2,2,q[4:8])
vi = q[8:10]
vf = q[10:12]
symbaut = WeightedAutomaton({'0':P0,'1':P1},vi,vf,ring=SR)

In [26]:
# Let's use the conjugate gradient method to see how high of a gap we can find for the string 0110 among
# 3-state generalized automata with coefficients in [-1,1]. Thanks to Patrick Lutz for suggesting this approach.
# Forcing the WA to be a PFA is a lot messier to do with minimize_constrained(), so let's not bother for this example.
# First, Sage only lets us _minimize_ stuff, so we have to consider the negative of the gap function:
theword = '01101'
neggap = -symbaut.gap(theword)
# The function to actually be minimized (plug in the values of x to the variables q)
f = lambda x: neggap.subs(dict(zip(q,x)))

In [27]:
# Pick our starting values to be ones I found from a brute-force search a while ago. That one gave 01101 a gap of ~0.0176.
# ***NOTE*** this may take some time to run, depending on your machine.
coeffs = minimize_constrained(f, [[-1,1]]*12, [1/2,1/2, # P0
                                               0,1/2,
                                               1/4,3/4, # P1
                                               1,0,
                                               1,0, # initial vector
                                               1,0], # final vector
                              algorithm='l-bfgs-b')
print(coeffs, -f([QQ(t) for t in coeffs]).n())

(0.5434950393537632, 0.9999999999029563, -0.13221469257774374, 0.5034272987915008, 0.5139988215364416, 0.5685278448936099, 1.0, 0.2559818931680029, 1.0, -0.08031545460682817, 1.0, -0.13316696286466193) 0.296835262992385
